## Tabela Silver para Dimenssão de Localização

#### Tratamento do Dataframe para obter informações desejadas

Aqui vamos filtrar apenas as colunas desejadas e realizar o tratamentod e tipagem e limpeza de dados necessárias

Vamos realizar alguns tratamentos

    Seleção de colunas necessárias
    Configurar colunas para o padrão
    Filtrando dados indesejados
    Alteração de tipagem de dados
    Removendo dados nulos
    Salvando arquivo de forma compactada

In [0]:
# Importando Bibliotecas para uso do Spark
from pyspark.sql import SparkSession # Import para inciar o spark
from pyspark.sql.types import * # Todos as configurações do SQL de tipagem no spark
from pyspark.sql.functions import * # Todas as funções de SQL do Spark
from pyspark.sql.window import Window

In [0]:
# Diretorio Bronze, com os dados brutos das causas dos acidentes
Bronze_Acidentes_Causas = '/Volumes/workspace/dw_acidentes/01_bronze/Acidentes_Causas/'

# Local para armazenar dimenssão
Diretorio_Silver = '/Volumes/workspace/dw_acidentes/02_silver'


In [0]:
# Lê os arquivos CSV descompactado
df_Acidentes_Causas = spark.read.parquet(Bronze_Acidentes_Causas)

In [0]:
# Selecionado apenas as colunas necessária para criar o modelo Dimenssional de Localização
df_localizacao = df_Acidentes_Causas.select(['municipio', 'uf', 'br', 'tipo_pista', 'km', 'tracado_via', 'uso_solo', 'sentido_via', 'regional', 'delegacia', 'uop', 'latitude', 'longitude'])

In [0]:
# Removendo Duplicadas para criar o campo chave
df_localizacao = df_localizacao.dropDuplicates()

In [0]:
# Criando a função de janela e ordenando todos as colunas aplicando linha a linha 
window = Window.orderBy('municipio', 'uf', 'br', 'tipo_pista', 'km', 'tracado_via', 'uso_solo', 'sentido_via', 'regional', 'delegacia', 'uop', 'latitude', 'longitude')

# Aplicando a função row_number e over para cada linha incluir o ID incremental.
df_localizacao = df_localizacao.withColumn(
    "id_localizacao",
    row_number().over(window)
)

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
# Selecionado apenas as colunas na ordem desejada
df_localizacao = df_localizacao.select(['id_localizacao', 'municipio', 'uf', 'br', 'tipo_pista', 'km', 'tracado_via', 'uso_solo', 'sentido_via', 'regional', 'delegacia', 'uop', 'latitude', 'longitude'])

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
# Alterando a tipagem para padrões necesários - Realizado "regexp_replace" para remover dados com "."" E substituir as "," por ".", por fim alterar os tipos para decimal
df_localizacao = df_localizacao.withColumn("km",regexp_replace(regexp_replace(col("km"), "\\.", ""), ",", ".").try_cast(DecimalType(18, 8)))
df_localizacao = df_localizacao.withColumn("latitude",regexp_replace(regexp_replace(col("latitude"), "\\.", ""), ",", ".").try_cast(DecimalType(18, 8)))
df_localizacao = df_localizacao.withColumn("longitude",regexp_replace(regexp_replace(col("longitude"), "\\.", ""), ",", ".").try_cast(DecimalType(18, 8)))

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


In [0]:
# Validando os campos para realizar os tratamentos
df_localizacao.printSchema()

root
 |-- id_localizacao: integer (nullable = false)
 |-- municipio: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- br: string (nullable = true)
 |-- tipo_pista: string (nullable = true)
 |-- km: decimal(18,8) (nullable = true)
 |-- tracado_via: string (nullable = true)
 |-- uso_solo: string (nullable = true)
 |-- sentido_via: string (nullable = true)
 |-- regional: string (nullable = true)
 |-- delegacia: string (nullable = true)
 |-- uop: string (nullable = true)
 |-- latitude: decimal(18,8) (nullable = true)
 |-- longitude: decimal(18,8) (nullable = true)



In [0]:
# Visualizando o primeiros registros para visualizar o resultado final
display(df_localizacao.limit(5))

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


id_localizacao,municipio,uf,br,tipo_pista,km,tracado_via,uso_solo,sentido_via,regional,delegacia,uop,latitude,longitude
1,ABADIA DE GOIAS,GO,60,Dupla,163.00000000,Reta,Sim,Decrescente,SPRF-GO,DEL01-GO,UOP02-DEL01-GO,-16.79103947,-49.50351925
2,ABADIA DE GOIAS,GO,60,Dupla,165.30000000,Reta,Não,Crescente,SPRF-GO,DEL01-GO,UOP02-DEL01-GO,-16.74020307,-49.41108495
3,ABADIA DE GOIAS,GO,60,Dupla,168.00000000,Reta;Declive,Sim,Crescente,SPRF-GO,DEL01-GO,UOP02-DEL01-GO,-16.72230000,-49.37460000
4,ABADIA DE GOIAS,GO,60,Dupla,169.00000000,Reta,Não,Decrescente,SPRF-GO,DEL01-GO,UOP02-DEL01-GO,-16.74760217,-49.42389607
5,ABADIA DE GOIAS,GO,60,Dupla,169.00000000,Reta,Sim,Crescente,SPRF-GO,DEL01-GO,UOP02-DEL01-GO,-16.72690000,-49.38220000


In [0]:
# Salvando o arquivo em parquet com compressão gzip 
df_localizacao.write.parquet(
    f"{Diretorio_Silver}/Dim_Localizacao", 
    mode="overwrite",         # Aqui defino para sempre sobrescrever o arquivo
    compression="gzip"        # Comando para salvar o arquivo com compactação
)

/databricks/python/lib/python3.11/site-packages/pyspark/sql/connect/expressions.py:1017: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(
